<a href="https://colab.research.google.com/github/AdrZx/ADS2002-fyan0039/blob/main/W5_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
def reg_model_accuracy(model, features, labels, randomnumb=42):
    '''Convenience function to fit an sklearn regression model and calculate the r2 score'''
    import numpy as np
    from sklearn.pipeline import make_pipeline
    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics import r2_score # import the score functions
    from sklearn.model_selection import train_test_split

    X_train, X_test, y_train, y_test = train_test_split(features, labels, random_state=randomnumb)
    pipe = make_pipeline(StandardScaler(), model)
    pipe.fit(X_train, y_train)  # apply scaling on training data
    y_pred= pipe.predict(X_test) # calculate the predicted values of the model for the test features

    rsq = np.round(r2_score(y_test, y_pred),3)

    return rsq

In [6]:
import pandas as pd
Boston = pd.read_csv('housing.csv')
BostonFeatures = Boston.drop(columns=['MEDV'])
BostonLabels = Boston['MEDV']
Boston

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273,21.0,393.45,6.48,22.0


Exercise 1

In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

X = BostonFeatures
y = BostonLabels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)

rf_regressor.fit(X_train, y_train)

y_pred = rf_regressor.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse:.2f}")


Mean Squared Error: 7.90


Exercise 2

In [59]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, VotingClassifier

X = Boston.drop('MEDV', axis = 1) # drop the target variable for the features
y = Boston['MEDV'] # create a target dataframe
# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.8, random_state=0)
# instantatiate the RFC with 100 ensemble members

num_features_to_select = 5
Best_Five = SelectKBest(score_func=f_regression, k=num_features_to_select)
X_train_selected = Best_Five.fit_transform(X_train, y_train)
X_test_selected = Best_Five.transform(X_test)

selected_feature_indices = Best_Five.get_support(indices=True)
selected_feature_names = BostonLabels[selected_feature_indices]
print("Selected Features:", selected_feature_names)

Selected Features: 2     34.7
5     28.7
9     18.9
10    15.0
12    21.7
Name: MEDV, dtype: float64


In [60]:
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)

rf_regressor.fit(X_train_selected, y_train)

y_pred = rf_regressor.predict(X_test_selected)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error with selected features: {mse:.2f}")

Mean Squared Error with selected features: 26.37


Exercise 3

In [67]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error

num_features_to_select = 5
rfe = RFE(estimator=rf_regressor, n_features_to_select=num_features_to_select)
X_train_rfe = rfe.fit_transform(X_train, y_train)
X_test_rfe = rfe.transform(X_test)

feature_ranking = rfe.ranking_

# selected_feature_indices = rfe.get_support(indices=True)
selected_feature_indices = np.where(feature_ranking == 1)[0]
selected_feature_names = BostonLabels[selected_feature_indices]
print("Selected Features:", selected_feature_names)

Selected Features: 0     24.0
5     28.7
7     27.1
10    15.0
12    21.7
Name: MEDV, dtype: float64


In [66]:
rf_regressor.fit(X_train_rfe, y_train)

y_pred = rf_regressor.predict(X_test_rfe)

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error with RFE-selected features: {mse:.2f}")

Mean Squared Error with RFE-selected features: 21.39


I acknowledge use the ChatGPT (https://chat.openai.com/) to research the information about the RandomForestRegressor, SelectKBest, and RFE.